In [1]:
import sys
sys.path.append('/home/sfang/windows/gitlab/stanleysfang/surveillance_2019_ncov/prod')

In [2]:
from google.cloud import bigquery
from BigQueryWrapper import QueryRunner, Loader, Extractor
import pandas as pd
import datetime
import re

In [3]:
project_id = 'stanleysfang'

In [4]:
client = bigquery.Client(project=project_id)

In [5]:
qr = QueryRunner(client=client)
loader = Loader(client=client)
extractor = Extractor(client=client)

In [6]:
max_results = 20
geo='us'

#### Load Data

In [7]:
confirmed = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')

In [8]:
confirmed.iloc[:, 0:15]

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20
0,NaN,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,0,0,0,0,0
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,0,0,0,0,0
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,0,0,0,0,0
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,0,0,0,0,0
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264,NaN,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,0,0,0,0,0
265,NaN,Western Sahara,24.215500,-12.885800,0,0,0,0,0,0,0,0,0,0,0
266,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,0,0,0,0,0
267,NaN,Zambia,-13.133897,27.849332,0,0,0,0,0,0,0,0,0,0,0


In [9]:
deaths.iloc[:, 0:15]

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20
0,NaN,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,0,0,0,0,0
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,0,0,0,0,0
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,0,0,0,0,0
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,0,0,0,0,0
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264,NaN,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,0,0,0,0,0
265,NaN,Western Sahara,24.215500,-12.885800,0,0,0,0,0,0,0,0,0,0,0
266,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,0,0,0,0,0
267,NaN,Zambia,-13.133897,27.849332,0,0,0,0,0,0,0,0,0,0,0


In [11]:
dt_cols = [datetime.datetime.strptime(dt, '%m/%d/%y').strftime('dt_%Y%m%d') for dt in list(confirmed.columns)[11:]]

In [12]:
confirmed_cols = ['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'county', 'province_state', 'country_region', 'latitude', 'longitude', 'combined_key'] + dt_cols
confirmed.columns = confirmed_cols

In [13]:
confirmed_schema = [
    ('UID', 'INT64'),
    ('iso2', 'STRING'),
    ('iso3', 'STRING'),
    ('code3', 'INT64'),
    ('FIPS', 'FLOAT64'),
    ('county', 'STRING'),
    ('province_state', 'STRING'),
    ('country_region', 'STRING'),
    ('latitude', 'FLOAT64'),
    ('longitude', 'FLOAT64'),
    ('combined_key', 'STRING'),
] + [(dt_col, 'INT64') for dt_col in dt_cols]

In [14]:
confirmed_load_job = loader.load_df(confirmed, 'stanleysfang.surveillance_2019_ncov.ts_2019_ncov_{geo}_confirmed_raw'.format(geo=geo), schema=confirmed_schema)

In [15]:
deaths_cols = confirmed_cols
deaths_cols.insert(11, 'population')
deaths.columns = deaths_cols

In [16]:
deaths_schema = confirmed_schema
deaths_schema.insert(11, ('population', 'INT64'))

In [17]:
deaths_load_job = loader.load_df(deaths, 'stanleysfang.surveillance_2019_ncov.ts_2019_ncov_{geo}_deaths_raw'.format(geo=geo), schema=deaths_schema)

In [18]:
for job in loader.job_history:
    job.result()
    
    bq_table = client.get_table(job.destination)
    df = client.list_rows(bq_table, max_results=max_results).to_dataframe()
    
    print(bq_table.full_table_id)
    print(df.head(max_results))

/home/sfang/anaconda3/envs/surveillance_2019_ncov/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the REST endpoint.
  """


stanleysfang:surveillance_2019_ncov.ts_2019_ncov_us_confirmed_raw
         UID iso2 iso3  code3     FIPS          county  province_state  \
0   84080001   US  USA    840  80001.0       Out of AL         Alabama   
1   84090001   US  USA    840  90001.0      Unassigned         Alabama   
2   84002013   US  USA    840   2013.0  Aleutians East          Alaska   
3   84002016   US  USA    840   2016.0  Aleutians West          Alaska   
4   84002060   US  USA    840   2060.0     Bristol Bay          Alaska   
5   84002068   US  USA    840   2068.0          Denali          Alaska   
6   84002100   US  USA    840   2100.0          Haines          Alaska   
7   84002105   US  USA    840   2105.0   Hoonah-Angoon          Alaska   
8   84002158   US  USA    840   2158.0        Kusilvak          Alaska   
9   84080002   US  USA    840  80002.0       Out of AK          Alaska   
10  84002230   US  USA    840   2230.0         Skagway          Alaska   
11  84090002   US  USA    840  90002.0      Un

/home/sfang/anaconda3/envs/surveillance_2019_ncov/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the REST endpoint.
  """


stanleysfang:surveillance_2019_ncov.ts_2019_ncov_us_deaths_raw
         UID iso2 iso3  code3     FIPS        county  province_state  \
0   84001045   US  USA    840   1045.0          Dale         Alabama   
1   84013305   US  USA    840  13305.0         Wayne         Georgia   
2   84090017   US  USA    840  90017.0    Unassigned        Illinois   
3   84080036   US  USA    840  80036.0     Out of NY        New York   
4   84037087   US  USA    840  37087.0       Haywood  North Carolina   
5   84038015   US  USA    840  38015.0      Burleigh    North Dakota   
6   84047179   US  USA    840  47179.0    Washington       Tennessee   
7   84048249   US  USA    840  48249.0     Jim Wells           Texas   
8   84048409   US  USA    840  48409.0  San Patricio           Texas   
9   84048465   US  USA    840  48465.0     Val Verde           Texas   
10  84005051   US  USA    840   5051.0       Garland        Arkansas   
11  84013175   US  USA    840  13175.0       Laurens         Georgia   
1

#### Dataprep

In [38]:
for metric in ['confirmed', 'deaths']:
    array_query = ''
    for dt_col in dt_cols:
        array_query = \
        """{array_query}
        STRUCT(PARSE_DATE('dt_%Y%m%d', '{dt_col}') AS dt, {dt_col} AS {metric}),
        """.format(array_query=array_query, dt_col=dt_col, metric=metric)
    array_query = re.sub('[,\n ]*$', '\n', array_query)
    
    query = \
    """
    SELECT dt, a.* EXCEPT(arr), {metric}
    FROM (
        SELECT
            city, province_state, country_region, combined_key, latitude, longitude,
            [{array_query}] AS arr
        FROM `stanleysfang.surveillance_2019_ncov.ts_2019_ncov_{geo}_{metric}_raw`
    ) a, UNNEST(arr)
    """.format(metric=metric, array_query=array_query, geo=geo)
    
    query_job = qr.run_query(query, destination_table='stanleysfang.surveillance_2019_ncov.ts_2019_ncov_{geo}_{metric}'.format(geo=geo, metric=metric), time_partitioning=True, partition_field='dt')
    query_job.result()
    
    bq_table = client.get_table(query_job.destination)
    df = client.list_rows(bq_table, max_results=max_results).to_dataframe()
    
    print(bq_table.full_table_id)
    print(df.head(max_results))


/home/sfang/anaconda3/envs/surveillance_2019_ncov/lib/python3.7/site-packages/ipykernel_launcher.py:23: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the REST endpoint.


stanleysfang:surveillance_2019_ncov.ts_2019_ncov_us_confirmed
            dt            city  province_state country_region  \
0   2020-02-04       Out of AL         Alabama             US   
1   2020-02-04      Unassigned         Alabama             US   
2   2020-02-04  Aleutians East          Alaska             US   
3   2020-02-04  Aleutians West          Alaska             US   
4   2020-02-04     Bristol Bay          Alaska             US   
5   2020-02-04          Denali          Alaska             US   
6   2020-02-04          Haines          Alaska             US   
7   2020-02-04   Hoonah-Angoon          Alaska             US   
8   2020-02-04        Kusilvak          Alaska             US   
9   2020-02-04       Out of AK          Alaska             US   
10  2020-02-04         Skagway          Alaska             US   
11  2020-02-04      Unassigned          Alaska             US   
12  2020-02-04        Wrangell          Alaska             US   
13  2020-02-04         Yakut

/home/sfang/anaconda3/envs/surveillance_2019_ncov/lib/python3.7/site-packages/ipykernel_launcher.py:23: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the REST endpoint.


stanleysfang:surveillance_2019_ncov.ts_2019_ncov_us_deaths
            dt          city  province_state country_region  \
0   2020-01-22          Dale         Alabama             US   
1   2020-01-22         Wayne         Georgia             US   
2   2020-01-22    Unassigned        Illinois             US   
3   2020-01-22     Out of NY        New York             US   
4   2020-01-22       Haywood  North Carolina             US   
5   2020-01-22      Burleigh    North Dakota             US   
6   2020-01-22    Washington       Tennessee             US   
7   2020-01-22     Jim Wells           Texas             US   
8   2020-01-22  San Patricio           Texas             US   
9   2020-01-22     Val Verde           Texas             US   
10  2020-01-22       Garland        Arkansas             US   
11  2020-01-22       Laurens         Georgia             US   
12  2020-01-22      Kootenai           Idaho             US   
13  2020-01-22    Evangeline       Louisiana             US

In [10]:
ts_2019_ncov_temp_query = ''

for dt_col in dt_cols:
    ts_2019_ncov_temp_query = \
    """{ts_2019_ncov_temp_query}
    SELECT
        PARSE_DATE('dt_%Y%m%d', '{dt_col}') AS dt,
        a.province_state, a.country_region, a.latitude, a.longitude,
        a.{dt_col} AS confirmed,
        b.{dt_col} AS deaths,
        c.{dt_col} AS recovered,
    FROM `stanleysfang.surveillance_2019_ncov.ts_2019_ncov_confirmed_raw` a
    LEFT JOIN `stanleysfang.surveillance_2019_ncov.ts_2019_ncov_deaths_raw` b
    ON IFNULL(a.province_state, '') = IFNULL(b.province_state, '') AND a.country_region = b.country_region
    LEFT JOIN `stanleysfang.surveillance_2019_ncov.ts_2019_ncov_recovered_raw` c
    ON IFNULL(a.province_state, '') = IFNULL(c.province_state, '') AND a.country_region = c.country_region
    
    UNION ALL
    """.format(ts_2019_ncov_temp_query=ts_2019_ncov_temp_query, dt_col=dt_col)

ts_2019_ncov_temp_query = re.sub('[(UNION ALL)\n ]*$', '\n', ts_2019_ncov_temp_query)

In [ ]:
SELECT
    
FROM (
    SELECT
        a.*, b.population
    FROM (
        SELECT
            COALESCE(a.dt, b.dt) AS dt,
            COALESCE(a.city, b.city) AS city,
            COALESCE(a.province_state, b.province_state) AS province_state,
            COALESCE(a.country_region, b.country_region) AS country_region,
            COALESCE(a.combined_key, b.combined_key) AS combined_key,
            COALESCE(a.latitude, b.latitude) AS latitude,
            COALESCE(a.longitude, b.longitude) AS longitude,
            IFNULL(a.confirmed, 0) AS confirmed,
            IFNULL(b.deaths, 0) AS deaths
        FROM `stanleysfang.surveillance_2019_ncov.ts_2019_ncov_us_confirmed` a
        FULL JOIN `stanleysfang.surveillance_2019_ncov.ts_2019_ncov_us_deaths` b
        ON a.dt = b.dt AND a.combined_key = b.combined_key
    ) a
    LEFT JOIN `stanleysfang.surveillance_2019_ncov.ts_2019_ncov_us_deaths_raw` b
    ON a.combined_key = b.combined_key
)

In [11]:
# print(ts_2019_ncov_temp_query)

In [12]:
ts_2019_ncov_query = \
"""
SELECT
    dt, province_state, country_region, latitude, longitude,
    confirmed, deaths, recovered,
    IFNULL(confirmed_new, confirmed) AS confirmed_new,
    IFNULL(deaths_new, deaths) AS deaths_new,
    IFNULL(recovered_new, recovered) AS recovered_new,
    TIMESTAMP(REGEXP_REPLACE(STRING(CURRENT_TIMESTAMP, "America/Los_Angeles"), r'[\+-][0-9]{2}$', '')) AS last_updated_ts
FROM (
    SELECT
        dt, province_state, country_region, latitude, longitude,
        confirmed, deaths, recovered,
        confirmed - LAG(confirmed) OVER(PARTITION BY province_state, country_region ORDER BY dt) AS confirmed_new,
        deaths - LAG(deaths) OVER(PARTITION BY province_state, country_region ORDER BY dt) AS deaths_new,
        recovered - LAG(recovered) OVER(PARTITION BY province_state, country_region ORDER BY dt) AS recovered_new
    FROM ({ts_2019_ncov_temp_query})
)
""".format(ts_2019_ncov_temp_query=ts_2019_ncov_temp_query)

In [13]:
job_config = bigquery.QueryJobConfig()

job_config.use_legacy_sql = False
job_config.destination = 'stanleysfang.surveillance_2019_ncov.ts_2019_ncov'
job_config.write_disposition = 'WRITE_TRUNCATE'
job_config.time_partitioning = bigquery.table.TimePartitioning(field='dt')
job_config.dry_run = False

In [14]:
query_job = client.query(ts_2019_ncov_query, job_config=job_config)

In [15]:
query_job.result()

#### Extract Table

In [16]:
job_config = bigquery.ExtractJobConfig()

job_config.destination_format = 'CSV'

In [17]:
extract_job = client.extract_table(
    'stanleysfang.surveillance_2019_ncov.ts_2019_ncov',
    'gs://surveillance_2019_ncov/ts_2019_ncov.csv',
    job_config=job_config
)

In [18]:
extract_job.result()